# Word count

Word count has been called the "Hello World" of big data. This makes sense because counting word occurences comes down to simple map and reduce operations. It turns out that suprisingly many of the more involved types of analysis we wish to perform on data can be broken down to operations very similar to word count.

Let us start with a small dataset of Shakespear quotes:

In [1]:
shakespear_data=spark.sparkContext.parallelize(["All that glitters is not gold", 
                                     "Love all, trust a few, do wrong to none",
                                    "To be or not to be that is the question",
                                    "my kingdom for a horse",
                                    "the world is a stage",
                                    "is this a dagger that I see before me",
                                    "nothing will come of nothing"])

In Spark we can perform wordcount in very few lines of code:

In [2]:
shakespear_data.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .collect()

[('glitters', 1),
 ('is', 4),
 ('gold', 1),
 ('Love', 1),
 ('do', 1),
 ('none', 1),
 ('question', 1),
 ('world', 1),
 ('stage', 1),
 ('this', 1),
 ('before', 1),
 ('of', 1),
 ('All', 1),
 ('that', 3),
 ('not', 2),
 ('all,', 1),
 ('trust', 1),
 ('a', 4),
 ('few,', 1),
 ('wrong', 1),
 ('to', 2),
 ('To', 1),
 ('be', 2),
 ('or', 1),
 ('the', 2),
 ('my', 1),
 ('kingdom', 1),
 ('for', 1),
 ('horse', 1),
 ('dagger', 1),
 ('I', 1),
 ('see', 1),
 ('me', 1),
 ('nothing', 2),
 ('will', 1),
 ('come', 1)]

We can add a few more operations to sort the list of words by occurance and only retrieve the top ten words:

In [3]:
shakespear_data.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .map(lambda x: (x[1], x)) \
             .sortByKey(False) \
             .map(lambda x: x[1]) \
             .take(10)

[('is', 4),
 ('a', 4),
 ('that', 3),
 ('not', 2),
 ('to', 2),
 ('be', 2),
 ('the', 2),
 ('nothing', 2),
 ('glitters', 1),
 ('gold', 1)]

In [4]:
def topWords(data, n=10):
    return data.flatMap(lambda line: line.lower().split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .map(lambda x: (x[1], x)) \
             .sortByKey(False) \
             .map(lambda x: x[1]) \
             .take(n)

In [12]:
nytaar=spark.sparkContext.textFile("gs://big-data-course-datasets/nytaar/").cache()

In [13]:
topWords(nytaar, n=12)

[('og', 2368),
 ('det', 1702),
 ('i', 1546),
 ('at', 1463),
 ('vi', 1450),
 ('er', 1300),
 ('', 1153),
 ('for', 1142),
 ('til', 985),
 ('de', 849),
 ('som', 847),
 ('har', 766)]

What if we want to know the count of a specific word? For example what word has the higher count: "prins" (prince) or "prinsesse" (princess)?

In [14]:
nytaar.flatMap(lambda line: line.lower().split(" ")) \
             .filter(lambda x: x=="prins" or x=="prinsesse") \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .map(lambda x: (x[1], x)) \
             .sortByKey(False) \
             .map(lambda x: x[1]) \
             .take(3)

[('prins', 38), ('prinsesse', 16)]